In [209]:
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [385]:
data_2020 = 'https://s3.amazonaws.com/parkerhiggins-nba-draft-bucket/19-20_MBB_StatsAndDraft.csv'
raw_2020_data = pd.read_csv(data_2020)

In [386]:
conf_data = pd.read_csv('https://s3.amazonaws.com/parkerhiggins-nba-draft-bucket/NCAA_conference_data_cleaned.csv')

In [387]:
conf_data.head()

,TEAM,CONF
0,Michigan,B10
1,Baylor,B12
2,Illinois,B10
3,Gonzaga,WCC
4,Iowa,B10


In [388]:
raw_2020_data.head()

,#,player,team,gp,mpg,fgm,fga,FG%,3PM,3PA,...,pf,orb,drb,rpg,apg,spg,bpg,ppg,Pk,Player1
0,47,Anthony Edwards,UGA,32,33.0,6.3,15.8,0.402,2.2,7.7,...,2.2,0.8,4.5,5.2,2.8,1.3,0.6,19.1,1.0,Anthony Edwards
1,1085,Patrick Williams,FSU,29,22.5,3.3,7.1,0.459,0.6,1.7,...,1.6,1.3,2.7,4.0,1.0,1.0,1.0,9.2,4.0,Patrick Williams
2,487,Isaac Okoro,AU,28,31.5,4.5,8.8,0.510,0.7,2.5,...,2.7,1.9,2.5,4.4,2.0,0.9,0.9,12.8,5.0,Isaac Okoro
3,164,Onyeka Okongwu,USC,28,30.6,6.2,10.1,0.616,0.0,0.1,...,2.7,3.3,5.4,8.6,1.1,1.2,2.7,16.2,6.0,Onyeka Okongwu
4,27,Obi Toppin,DAY,31,31.6,7.9,12.5,0.633,1.0,2.6,...,1.6,1.2,6.4,7.5,2.2,1.0,1.2,20.0,8.0,Obi Toppin


In [389]:
X.columns

Index(['ppg', 'mpg', 'rpg', 'apg', 'spg', 'gp', 'tov'], dtype='object')

In [603]:
numerical_cols = raw_2020_data.dtypes[raw_2020_data.dtypes != 'object'].index.tolist()

numerical_df = raw_2020_data[numerical_cols]
numerical_df.index = raw_2020_data['#']

numerical_df['draft_status'] = numerical_df['Pk'].apply(lambda x: True if x <= 60 else False)

# selected_features = ['ppg','mpg','rpg','apg','spg','gp','tov','draft_status']
selected_features = ['ppg','rpg','apg','spg','tov','draft_status','FG%','3P%','FT%']
data_df = numerical_df[selected_features]

# X = numerical_df.drop(columns=['Pk','#','pf','draft_status'])
X = data_df.drop(columns=['draft_status'])
y = data_df['draft_status']

# scaler = StandardScaler()
# X = scaler.fit_transform(X)

ipykernel_launcher:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [604]:
len(selected_features)-1

8

In [632]:
model_filename = './svm_2007_2019.sav'
loaded_model = pickle.load(open(model_filename, 'rb'))

In [633]:
predictions = loaded_model.predict(X)
results = loaded_model.score(X,y)
print(results)

0.7405


In [634]:
draft_prob = loaded_model.predict_proba(X)

In [635]:
draft_prob_true = []
for prob in draft_prob:
    draft_prob_true.append(prob[1])

In [636]:
numerical_df['draft_prob_true'] = draft_prob_true
numerical_df['predictions'] = predictions

In [637]:
# numerical_df[numerical_df['Pk'] <= 60].sort_values(by='draft_prob_true',ascending=False)

In [638]:
# numerical_df.sort_values(by=['draft_prob_true'],ascending=False).head(2S)

In [639]:
acc_score = accuracy_score(y,predictions)
matrix = confusion_matrix(y,predictions)
results = pd.DataFrame({"Prediction": predictions, "Actual": y}).reset_index(drop=True)
report = classification_report(y,predictions,zero_division=True)

print(acc_score,'\n')
print(matrix,'\n')
print(report,'\n')

# numerical_df[['draft_status','draft_prob_true']].sort_values(by='draft_prob_true',ascending=False).head(39).value_counts().groupby('draft_status').count()
numerical_df[['draft_prob_true','draft_status']].sort_values(by='draft_prob_true',ascending=False).head(39).value_counts().groupby('draft_status').sum()

0.7405 

[[1447  514]
 [   5   34]] 

              precision    recall  f1-score   support

       False       1.00      0.74      0.85      1961
        True       0.06      0.87      0.12        39

    accuracy                           0.74      2000
   macro avg       0.53      0.80      0.48      2000
weighted avg       0.98      0.74      0.83      2000
 



draft_status
False    30
True      9
dtype: int64

In [640]:
# numerical_df[['draft_status','draft_prob_true']].sort_values(by='draft_prob_true',ascending=False).head(39).value_counts().groupby('draft_status').count()
# numerical_df[['draft_status','draft_prob_true']].groupby(by='draft_status',axis=1,as_index=False).count()